In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp downloaderlabeler
# all_local

In [ ]:
import ipywidgets as widgets
from ipywidgets import Button, Layout
import pickle
import json
from steroidsornot.submissionshandler import SubmissionsHandler
from steroidsornot.database import Session
from steroidsornot.submission import Submission

In [ ]:
from fastcore.test import *

In [ ]:
class DownloaderLabeler(object):
    def __init__(self, session):
        self.session = session
        self.unlabeled_submissions = session.query(Submission).filter_by(label=None)

        
        self._setup_buttons()
        
        self._main_loop()
        
    def _main_loop(self):
        for submission in self.unlabeled_submissions:
            data = json.loads(submission.data)
            print(submission.id, end=' ')
        
        self.session.close()

        
    def _setup_buttons(self):
        self.natural_button = Button(description='natural', layout=Layout(border = '2px solid green'))
        self.uncertain_button = Button(description='uncertain', layout=Layout(border = '2px solid brown'))
        self.steroids_button = Button(description='steroids', layout=Layout(border = '2px solid red'))
        self.irrelevant_button = Button(description='irrelevant', layout=Layout(border = '2px solid khaki'))
        self.box = widgets.HBox([self.natural_button,
                                 self.uncertain_button,
                                 self.steroids_button,
                                 self.irrelevant_button,
                                ])
        display(self.box)       

In [ ]:
downloader_labeler = DownloaderLabeler(Session())